In [70]:
from pulp import *
import pandas as pd

# The Spotify Dataset

The goal of this project is to create an optimal subplaylist of a bigger playlist, maximizing some metrics related to how good the tracks are, subject to maximum duration and possibly other constraints.

We'll use the Spotify Playlist [For Those About to Rock](https://open.spotify.com/playlist/5TUxgTIxzLbLVh7RUf9V8i?si=9c057405f51747b8) created by [Rafaella Bellerini](https://github.com/rafaballerini) with more than a thousand rock songs.

Data was colected through Spotify API, which provides the following metrics for each track (descriptions from [API documentation]((https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features))):

- Track Popularity:
    
    The value will be between 0 and 100, with 100 being the most popular.

- Artist Popularity:

    The artist's popularity is calculated from the popularity of all the artist's tracks, and also ranges from 0 to 100.

- Duration:

    The track length (in minutes)

- Danceability:

    Describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0 is least danceable and 1 is most danceable.

- Energy:

    A measure from 0 to 1 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy.

- Acousticness:

    A confidence measure from 0 to 1 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.

- Instrumentalness:

    Predicts whether a track contains no vocals. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1, the greater likelihood the track contains no vocal content

- Liveness:

    Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live.

- Valence:

    A measure from 0 to 1 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

In [71]:
df = pd.read_csv("../data/for_those_about_to_rock.csv")
df.head()

,track_name,artist,track_pop,artist_pop,duration,danceability,energy,acousticness,instrumentalness,liveness,valence,link
0,Sweet Child O' Mine,Guns N' Roses,84,82,5.908667,0.454,0.910,0.0866,0.09960,0.1160,0.629,open.spotify.com/track/7o2CTH4ctstm8TNelqjb51
1,Welcome To The Jungle,Guns N' Roses,80,82,4.533767,0.447,0.954,0.0235,0.40300,0.2980,0.331,open.spotify.com/track/0bVtevEgtDIeRjCJbK3Lmv
2,Paradise City,Guns N' Roses,81,82,6.760667,0.273,0.952,0.0169,0.01110,0.1420,0.472,open.spotify.com/track/3YBZIN3rekqsKxbJc9FZko
3,November Rain,Guns N' Roses,0,82,8.958433,0.197,0.629,0.0165,0.27900,0.1250,0.221,open.spotify.com/track/53968oKecrFxkErocab2Al
4,Knockin' On Heaven's Door,Guns N' Roses,0,82,5.600000,0.486,0.747,0.0203,0.00607,0.0992,0.368,open.spotify.com/track/7gXdAqJLCa5aYUeLVxosOz


# Problem Definition

The goal is to select an optimal subplaylist out of hundreds of tracks in Rafaella's playlist. A natural constraint is a limited duration since the whole playlist has over 24 hours of rock.

Another natural question is: "What is an optimal playlist?"

We can use any of the above-described audio features, as a maximization metric, but here we'll define "optimal" as "most popular" using track and artist popularity.

As additional restrictions, we can force the solver to include tracks of selected artists to match the user's tastes. Here we're going to force the inclusion of Guns N'Roses and AC/DC with at least three songs each.

The audio features not used for maximization can be used as contraints in the following way: The selected tracks must have an average metric above (or below) a given threshold. Here we're going to add a minumum energy and valence threshold trying to select energetic and happy songs.

So in natural language, our problem is:

**Sample tracks from a given playlist, maximizing track/artist popularity**

Subject to:

- **Sample duration less than given time limit**

- **At least three tracks by Guns N'Roses**

- **At least three tracks by AC/DC**

- **Average energy above given threshold**

- **Average valence above given threshold**


# Mathematical formulation

Let $x_i$ be the tracks binary decision variables, 

$d_i$ the duration of track $i$, 

$ap_i$ the artist popularity, 

$tp_i$ the track popularity,

$e_i$ the track energy, and

$v_i$ the track valence.



and $F=\{x_1^{F},\ldots,x_{N_F}^{(F)}\}$ the set of favorite artists tracks (Guns N' Roses and AC/DC).

The problem is:

$$\text{Maximize} \sum_{i=1}^{N}(ap_i+tp_i)x_i$$

$$ \text{subject to} \sum_{i=1}^{N}d_ix_i\leq D,$$

$$ \sum_{j=1}^{N_F}x_j^{(F)}\geq M,$$

$$\sum_{i=1}^{N}(v_i-V)x_i\geq 0,$$

$$\sum_{i=1}^{N}(e_i-E)x_i\geq 0,$$

$$\text{ and } x_i\in \{0,1\}\text{ for } i = 1,\ldots N$$

being $D$ a given time limit, $M$ the minimum favorite artist tracks, $N$ the total number of tracks, $N_F$ total favorite artists tracks, $V$ a valence threshold, $E$ an energy threshold.

Explaining energy and valence constraits:

We want selected tracks to have an avarege energy greater then $E$, so originally the constraint is:

$$\dfrac{1}{\sum_{i}x_i}\sum_{i=1}^Ne_ix_i\geq E,$$

which implies: 

$$\sum_{i=1}^Ne_ix_i\geq E\sum_{i=1}^Nx_i$$

such expression can be easily manipulated to get the problem constraint. The procedure for valence is analogous.

We're going to solve two variations of this problem, one excluding favorite artist constraint and the other including it.

# Using PuLP to solve the problem

The following steps define and solve the problem using Python [PuLP library](https://coin-or.github.io/pulp/)

Reading dataframe:

In [72]:
df = pd.read_csv("../data/for_those_about_to_rock.csv")

Defining variables as track name and artists:

In [73]:
df['duration'] = round(df.duration,2)
df['name'] = df['track_name']+"_"+df['artist']
df['name'] = df['name'].apply(lambda x:x.replace(" ","_"))

# Binary variables with lower bound = 0
vars = LpVariable.dict("",df.name,0,None,LpBinary)

df['varnames'] = df['name'].apply(lambda x:vars[x].getName())

Creating duration, popularity, energy and valence dictionaries:

In [75]:
durations = df[['name','duration']].set_index("name").to_dict()['duration']
artist_pop = df[['name','artist_pop']].set_index("name").to_dict()['artist_pop']
track_pop = df[['name','track_pop']].set_index("name").to_dict()['track_pop']
energy = df[['name','energy']].set_index("name").to_dict()['energy']
valence = df[['name','valence']].set_index("name").to_dict()['valence']

Defining thresholds:

In [76]:
# Max duration: 30 minutes
D = 30

# At least 3 tracks by each favorite artists
M = 3
favorite = ["AC_DC", "Guns_N'_Roses"]

# Valence threshold: 0.75
V = 0.75

#Energy Threshold: 0.75
E = 0.75

Defining the problems as a Maximization LP:

**Problem 1: Excluding favorite artist constraint;**

**Problem 2: All constraints included.**

In [77]:
prob1 = LpProblem("Optimal_Subplaylist_Problem_1", LpMaximize)

**Objective Function:** Sum of artist/track popularity of choosen songs

In [78]:
prob1 += (lpSum([vars[n]*(artist_pop[n]+track_pop[n]) for n in df.name]),
    "Sum_of_Popularity",)

**Constraints:**

In [79]:
# Max duration
prob1 += (
      lpSum([vars[n]*durations[n] for n in df.name]) <= D,
      "Sum_of_durations_(minutes)",
  )

# Minimum average valence:
prob1 += (
      lpSum([vars[n]*(valence[n]-V) for n in df.name]) >= 0,
      "Average_Valence",
  )

# Minimum average energy:
prob1 += (
      lpSum([vars[n]*(energy[n]-E) for n in df.name]) >= 0,
      "Average_Energy",
  )


Solving problem 1:

In [80]:
prob1.solve()
print("Prob1 Status:", LpStatus[prob1.status])

Prob1 Status: Optimal


## Problem 1 solution

Retrieving information of selected songs (by problem 1):

In [82]:
selected_vars = []
for v in prob1.variables():
  if v.varValue != 0:
    selected_vars.append(v.name)#, "=", v.varValue)
sol1 = df[df['varnames'].isin(selected_vars)][['track_name','artist','track_pop','duration','energy','valence','link']].reset_index().drop(['index'],axis=1)
sol1

,track_name,artist,track_pop,duration,energy,valence,link
0,Big Me,Foo Fighters,68,2.21,0.7080,0.8460,open.spotify.com/track/6pb5BBnIM5IM7R1cqag6rE
1,Eruption,Van Halen,50,1.71,0.7570,0.5210,open.spotify.com/track/3lW0MLws0srqqR3DRRPLZp
2,Speak to Me,Pink Floyd,62,1.09,0.0195,0.0313,open.spotify.com/track/574y1r7o2tRA009FW0LE7v
3,"Another Brick in the Wall, Pt. 3",Pink Floyd,58,1.24,0.4720,0.4480,open.spotify.com/track/5A7eooPKJHtr0UJmatjH4a
4,Immigrant Song - Remaster,Led Zeppelin,80,2.44,0.9320,0.6190,open.spotify.com/track/78lgmZwycJ3nzsdgmPPGNx
5,Hound Dog,Elvis Presley,68,2.27,0.7560,0.9490,open.spotify.com/track/64Ny7djQ6rNJspquof2KoX
6,Dancing Shoes,Arctic Monkeys,63,2.35,0.8890,0.8520,open.spotify.com/track/0hAMkY2kwdXPPDfQ1e3BmJ
7,Blitzkrieg Bop - 2016 Remaster,Ramones,75,2.24,0.7830,0.9020,open.spotify.com/track/4KcH1ZRV2W1q7Flq0QqC76
8,What Happened to You?,The Offspring,55,2.20,0.9430,0.9250,open.spotify.com/track/3U8LpjEbSlUQusIifQ3yBp
9,Tush - 2006 Remaster,ZZ Top,70,2.30,0.8850,0.7880,open.spotify.com/track/6zGDIDjfDkPyNxrEERO3XG


In [86]:
# Sanity check
print("Total duration:",sol1.duration.sum())
print("Average Valence:",sol1.valence.mean())
print("Average Energy:",sol1.energy.mean())

Total duration: 29.849999999999998
Average Valence: 0.7558199999999999
Average Energy: 0.7533666666666665


## Problem 2 solution

In [88]:
#Favorite Artists contraint:
prob2 = prob1.copy()
for fav in favorite:
  prob2 += (
        lpSum([vars[n] for n in df.name 
                    if fav in vars[n].getName()]) >= M,
              f"{fav}_Constraint",
  )
prob2.solve()
print("Prob2 Status:", LpStatus[prob2.status])

selected_vars = []
for v in prob2.variables():
  if v.varValue != 0:
    selected_vars.append(v.name)#, "=", v.varValue)
sol2 = df[df['varnames'].isin(selected_vars)][['track_name','artist','track_pop','duration','energy','valence','link']].reset_index().drop(['index'],axis=1)
sol2

Prob2 Status: Optimal


,track_name,artist,track_pop,duration,energy,valence,link
0,Attitude,Guns N' Roses,0,1.45,0.984,0.613,open.spotify.com/track/2iHQl54hUocY1JSdtISccd
1,Used To Love Her,Guns N' Roses,48,3.19,0.823,0.921,open.spotify.com/track/267BsRG0aX8LfeHsvCOdFA
2,Mr. Brownstone,Guns N' Roses,59,3.76,0.940,0.575,open.spotify.com/track/4DnEyHNO8MdhFYFrDq73BV
3,Rock or Bust,AC/DC,64,3.05,0.825,0.560,open.spotify.com/track/2vyae4DvSU2gs8OMOGwXF2
4,War Machine,AC/DC,63,3.16,0.718,0.531,open.spotify.com/track/5YnNWHKCJaJKwbagXDb5Uf
5,You Shook Me All Night Long,AC/DC,83,3.50,0.767,0.755,open.spotify.com/track/2SiXAy7TuUkycRVbbWDEpo
6,Big Me,Foo Fighters,68,2.21,0.708,0.846,open.spotify.com/track/6pb5BBnIM5IM7R1cqag6rE
7,"Another Brick in the Wall, Pt. 3",Pink Floyd,58,1.24,0.472,0.448,open.spotify.com/track/5A7eooPKJHtr0UJmatjH4a
8,Hound Dog,Elvis Presley,68,2.27,0.756,0.949,open.spotify.com/track/64Ny7djQ6rNJspquof2KoX
9,Song 2 - 2012 Remaster,Blur,80,2.02,0.789,0.918,open.spotify.com/track/1FTSo4v6BOZH9QxKc3MbVM


In [90]:
# Sanity check
print("Total duration:",sol2.duration.sum())
print("Average Valence:",sol2.valence.mean())
print("Average Energy:",sol2.energy.mean())

Total duration: 29.959999999999997
Average Valence: 0.7534166666666667
Average Energy: 0.7801666666666667


# Discussion and conclusion

We can see that selecting popular songs, subject to minimum valence and energy and limited time, does not necessarily match my tastes. But with additional constraints, we can force the appearance of these artists without losing control of the other constraints, i.e, in problem 1 we got popular and energetic tracks that fit 30 minutes, but in problem 2 we get all of that plus my favorite rock bands.

There are a lot of other possibilities of problems, including changing the maximization metric to another one and playing around with constraints and audio features. One can make a subplaylist maximizing danceability, subject to minimum popularity and acousticness for example. Future work would be to implement a web interface where the user can input a Spotify playlist, create its optimization problem, solve it and get a subplaylist!